In [1]:
#*************************************************************************************
#Creator - Shreejit Gajanan Deshmukh, Venkata Sai Advaith Kandiraju                  *
#PRCV spring 23' - Final Project Part 3                                              *
#Compiling lane detection, vehicle detection, driving assistance in video generation *
#*************************************************************************************

In [2]:
#*************************************************************************************
#Using the lane line detection and object detection and driving assitance code block,*
#this section computes video driving assistance by compiling the results from both   *
#into one. The output is the assited driving video which can be used by the driver.  *
#*************************************************************************************

In [ ]:
#Cloning the repo for lane data
!git clone https://github.com/udacity/CarND-LaneLines-P1.git

#Manually moved the test and train data outside in the jupyternotebook folder for easier access

In [3]:
#Part 3 - Compilation
#Importing libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from ultralytics import YOLO
import cvzone
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [4]:
#Loading the YOLOv8 model
model = YOLO("yolov8n.pt")

In [5]:
#Functions from lane detection
#Function to get vertices of the polygon where our lanes of interest lie in each frame.
def get_vertices(image):
    rows, cols = image.shape[:2] #excluding channel column
    
    #Vertices of the polygon, obtained with experimentation in each frame
    bottom_left  = [cols*0.15, rows]
    top_left     = [cols*0.45, rows*0.6]
    bottom_right = [cols*0.95, rows]
    top_right    = [cols*0.55, rows*0.6] 
    
    vertices = np.array([bottom_left, top_left, top_right, bottom_right], dtype = np.int32)
    
    #Ensuring proper data type for future use
    vertices = vertices.reshape((-1, 1, 2)).astype(np.int32)
    return vertices

In [6]:
#Function to extract the region where our lanes lie for further processing.
def masking(img, vertices):
    #mask of image shape
    mask = np.zeros_like(img)
    
    #defining mask for RGB or a single channel input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        
        #For region of interest
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
#     print(ignore_mask_color)
    
    cv2.fillPoly(mask, pts = [vertices], color = ignore_mask_color)
    
    #Masking the image to only keep the region of interest using bitwise and
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [7]:
#Function which draws red lines given a list of line endpoints and the frame image
def draw_lines(img, lines, color = [255, 0, 0], thickness = 10):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

In [8]:
#This function extracts the right and left lanes of our car using the slope concept. i.e. it is assumed that the
#left lane has positive slope and the right lane has negative slope
def slope_lines(image, lines):
    img = image.copy()
    
    #The vertices of polygon encompassing our lanes
    poly_vertices = []
    
    #For selecting vertices in order, used later
    order = [0,2,3,1]
    
    #For storing left and right lane points
    left_lines = []
    right_lines = []
    
    #segregating lines based on slopes
    for line in lines:
        for x1, y1, x2, y2 in line:
            #vertical lines
            if x1 == x2:
                pass
            else:
                m = (y2 - y1) / (x2 - x1)
                c = y1 - m*x1
                
                if m < 0:
                    left_lines.append((m,c))
                elif m >=0:
                    right_lines.append((m,c))
    
    #taking mean line from the cluster 
    left_line = np.mean(left_lines, axis = 0)
    right_line = np.mean(right_lines, axis = 0)
                
    #Drawing lines on our image using the mean lines calculated above
    for slope, intercept in [left_line, right_line]:
        
        rows, cols = image.shape[:2]
        
        y1 = int(rows)
        y2 = int(rows*0.6)
        
        x1=int((y1-intercept)/slope)
        x2=int((y2-intercept)/slope)
        
        poly_vertices.append((x1, y1))
        poly_vertices.append((x2, y2))
        
        draw_lines(img, np.array([[[x1,y1,x2,y2]]]))
    
    #To correctly select the polygon between lanes
    poly_vertices = [poly_vertices[i] for i in order]
    
    cv2.fillPoly(img, pts = np.array([poly_vertices], dtype = np.int32), color = (0,255,0))
    
    #Returning the mixed lanes added and our frame from camera
    return cv2.addWeighted(image,0.7,img,0.4,0.)

In [9]:
#Function for returning merged lines and image frames
def weighted_img(img, initial_img, alp = 0.1, beta = 1., gam = 0.):
    lines_edges = cv2.addWeighted(initial_img, alp, img, beta, gam)
    return lines_edges

In [10]:
#Function to extract lines from the points of lane
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength = min_line_len, maxLineGap = max_line_gap)
    
    #initializing a blank image to draw lines extracted from hough transform
    line_image = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
    
    #drawing lines on blank image
    line_image = slope_lines(line_image, lines)
    return line_image

In [11]:
#Lane detection pipeline
def lane_detection(image):
    #Converting to grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Smoothening the image using Gaussian blur
    gaus_blur = cv2.GaussianBlur(gray_image, (5,5), 0)
    
    #Finding the edges using canny edge detector
    canny_img = cv2.Canny(gaus_blur, 180, 240)
    
    #Masking the image to extract the region of interest
    masked_img = masking(img = canny_img, vertices = get_vertices(image))
    
    #Finding the lane lines using Hough transform
    lane_lines = hough_lines(img = masked_img, rho = 1, theta = np.pi/180, threshold = 20, min_line_len = 20, max_line_gap = 180)
#     print(len(lane_lines))
    
    #Drawing lines on lanes
    out_frame = weighted_img(img = lane_lines, initial_img = image, alp = 0.8, beta = 1., gam = 0.)
    
    return out_frame

In [12]:
#Function for object detection and driving assistance
def driving_assist(image):
    #Getting the recognized objects with bounding boxes and other YOLO outputs
    results = model(image)
    
    #number of cars and its position wrt center of the car
    right = 0
    left = 0
    ahead = 0

    #Flag for raising alarm if a car is close
    alert_flag = 0
    close_flag = 0
    close_left = 0
    close_right = 0
    close_ahead = 0
    
    #Loop through detections
    for r in results:
        boxes = r.boxes
    for box in boxes:
        cls = int(box.cls[0])
        #Only proceeding if identified class is road vehicle
        if (cls == 1 or cls == 2 or cls == 3 or cls == 5 or cls == 7):
            #Finding co-ordinates of the bounding boxes for later operations
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            #width and height of the boxes
            w, h = x2 -x1, y2 - y1

            #Drawing boxes around recognized objects
            bbox = x1, y1, w, h
            cvzone.cornerRect(image, bbox)

            #finding centers of each box to segragate
            center = ((x1 + x2) / 2, (y1 + y2) / 2)

            #Finding area to compute distance from our vehicle
            area = w * h

    #         print(center)
    #         print(area)

            frame_area = image.shape[0] * image.shape[1]

            #If the car area is beyond a limit it's too close, we will raise a flag for the driver
            #With experimentation current area pecentage is 2% for close vehicles.
            if (area > (frame_area*2) / 100):
                close_flag = 1
                alert_flag = 1
            else:
                close_flag = 0

            #Segregating
            ahead_tolerance = (image.shape[1] * 3) / 100
            if center[0] > image.shape[1] / 2 + ahead_tolerance:
                right += 1
                if(close_flag == 1):
                    close_right += 1
            if center[0] < image.shape[1] / 2 - ahead_tolerance:
                left += 1
                if(close_flag == 1):
                    close_left += 1
            if center[0] > image.shape[1] / 2 - ahead_tolerance and center[0] < image.shape[1] / 2 + ahead_tolerance:
                ahead += 1
                if(close_flag == 1):
                    close_ahead += 1

    #         print(left)
    #         print(ahead)
    #         print(close_left)
    #         print(close_right)
        else:
            pass

    #Adding information from frames to image
    text_far1 = f"Vehicles on left - {left}|| Vehicles on right - {right}"
    text_far2 = f"Vehicles infront - {ahead}"
    text_near_sides = f"Vehicles near on left - {close_left} || Vehicles near on right - {close_right}"
    text_front = "ALERT - Vehicle ahead -- SLOW DOWN"

    #If close cars are there then alert the driver
    if(alert_flag == 1):
        cv2.putText(image, text_near_sides, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
        if(close_ahead > 0):
            cv2.putText(image, text_front, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0))

    #Normal cars count
    cv2.putText(image, text_far1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255))
    cv2.putText(image, text_far2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255))
    return image

In [13]:
#Compiling the two
def comp(image):
    try:
        frame = lane_detection(image)
    except:
        print("Slope for this frame could not be detected")
        return image
    
    out_frame = driving_assist(frame)
    return out_frame

In [13]:
#Creating empty video to write
out_video = './solidWhiteRight.mp4'

#Loading our video
in_clip = VideoFileClip("test_videos/solidWhiteRight.mp4")

#Sending each frame through our pipeline
out_clip = in_clip.fl_image(comp)

#Display progress
%time out_clip.write_videofile(out_video, audio = False)


0: 384x640 4 cars, 79.9ms
Speed: 1.0ms preprocess, 79.9ms inference, 9.6ms postprocess per image at shape (1, 3, 640, 640)


Moviepy - Building video ./solidWhiteRight.mp4.
Moviepy - Writing video ./solidWhiteRight.mp4



t:   0%|                                                                             | 0/221 [00:00<?, ?it/s, now=None]
0: 384x640 4 cars, 10.1ms
Speed: 0.0ms preprocess, 10.1ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 4 cars, 8.0ms
Speed: 0.0ms preprocess, 8.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 2 trucks, 6.3ms
Speed: 0.9ms preprocess, 6.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
t:   2%|█▏                                                                   | 4/221 [00:00<00:05, 37.78it/s, now=None]
0: 384x640 3 cars, 1 truck, 9.9ms
Speed: 1.0ms preprocess, 9.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 8.7ms
Speed: 1.0ms preprocess, 8.7ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

t:  24%|████████████████▎                                                   | 53/221 [00:01<00:06, 27.24it/s, now=None]
0: 384x640 2 cars, 1 truck, 14.4ms
Speed: 0.0ms preprocess, 14.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 truck, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 10.8ms
Speed: 1.0ms preprocess, 10.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
t:  25%|█████████████████▏                                                  | 56/221 [00:01<00:06, 26.75it/s, now=None]
0: 384x640 2 cars, 1 truck, 12.5ms
Speed: 0.0ms preprocess, 12.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 truck, 5.5ms
Speed: 0.9ms preprocess, 5.5ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 1 truck, 12.5ms
Speed: 1.2ms preprocess, 12.5ms inference, 2.0ms postprocess per image a

t:  46%|██████████████████████████████▌                                    | 101/221 [00:03<00:04, 25.25it/s, now=None]
0: 384x640 2 cars, 1 truck, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 truck, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 truck, 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  47%|███████████████████████████████▌                                   | 104/221 [00:03<00:04, 25.07it/s, now=None]
0: 384x640 4 cars, 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 16.6ms
Speed: 0.0ms preprocess, 16.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 1 truck, 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1,

Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 13.6ms
Speed: 1.0ms preprocess, 13.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
t:  69%|██████████████████████████████████████████████                     | 152/221 [00:05<00:02, 26.00it/s, now=None]
0: 384x640 4 cars, 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 13.9ms
Speed: 0.0ms preprocess, 13.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  70%|██████████████████████████████████████████████▉                    | 155/221 [00:05<00:02, 25.65it/s, now=None]
0: 384x640 4 cars, 11.5ms
Speed: 1.0ms prepr

t:  90%|████████████████████████████████████████████████████████████▋      | 200/221 [00:07<00:00, 25.32it/s, now=None]
0: 384x640 5 cars, 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 12.0ms
Speed: 0.0ms preprocess, 12.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  92%|█████████████████████████████████████████████████████████████▌     | 203/221 [00:07<00:00, 25.06it/s, now=None]
0: 384x640 6 cars, 13.0ms
Speed: 0.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 18.1ms
Speed: 0.0ms preprocess, 18.1ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
t:  93%|██████████

Moviepy - Done !
Moviepy - video ready ./solidWhiteRight.mp4
CPU times: total: 1.2 s
Wall time: 8.68 s


In [14]:
#Creating empty video to write
out_video = './solidYellowLeft.mp4'

#Loading our video
in_clip = VideoFileClip("test_videos/solidYellowLeft.mp4")

#Sending each frame through our pipeline
out_clip = in_clip.fl_image(comp)

#Display progress
%time out_clip.write_videofile(out_video, audio = False)


0: 384x640 1 car, 22.6ms
Speed: 1.9ms preprocess, 22.6ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


Moviepy - Building video ./solidYellowLeft.mp4.
Moviepy - Writing video ./solidYellowLeft.mp4



t:   0%|                                                                             | 0/681 [00:00<?, ?it/s, now=None]
0: 384x640 1 car, 21.0ms
Speed: 2.0ms preprocess, 21.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 17.1ms
Speed: 0.9ms preprocess, 17.1ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
t:   0%|▎                                                                    | 3/681 [00:00<00:24, 27.77it/s, now=None]
0: 384x640 1 car, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 16.6ms
Speed: 2.9ms preprocess, 16.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:   1%|▌                                                                    | 6/681 [00:00<00:26, 25.60it/s, now=None]
0: 384x640 1 car, 16.0ms
Sp

t:  15%|█████████▉                                                          | 99/681 [00:03<00:23, 24.77it/s, now=None]
0: 384x640 1 car, 1 truck, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 17.0ms
Speed: 0.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 truck, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  15%|██████████                                                         | 102/681 [00:04<00:24, 23.83it/s, now=None]
0: 384x640 3 cars, 1 truck, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 truck, 19.0ms
Speed: 2.0ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 2 trucks, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shap

t:  29%|███████████████████▏                                               | 195/681 [00:08<00:19, 24.90it/s, now=None]
0: 384x640 1 truck, 1 kite, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 truck, 1 kite, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 truck, 15.0ms
Speed: 0.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  29%|███████████████████▍                                               | 198/681 [00:08<00:19, 25.17it/s, now=None]
0: 384x640 1 truck, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 truck, 1 kite, 12.5ms
Speed: 1.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 truck, 1 kite, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 3.0ms postprocess per image at sha

...

Speed: 1.1ms preprocess, 13.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 14.1ms
Speed: 1.0ms preprocess, 14.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 14.1ms
Speed: 0.9ms preprocess, 14.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
t:  58%|██████████████████████████████████████▊                            | 395/681 [00:15<00:09, 28.79it/s, now=None]
0: 384x640 (no detections), 13.8ms
Speed: 1.0ms preprocess, 13.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 13.1ms
Speed: 1.0ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
t:  58%|███████████████████████████████████████▏                           | 398/681 [00:15<00:09, 28.75it/s, now=None]


0: 384x640 (no detections), 14.0ms
Speed: 0.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 10.2ms
Speed: 1.0ms preprocess, 10.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 12.4ms
Speed: 1.0ms preprocess, 12.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
t:  65%|███████████████████████████████████████████▋                       | 444/681 [00:17<00:07, 29.93it/s, now=None]
0: 384x640 (no detections), 11.7ms
Speed: 0.0ms preprocess, 11.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 11.5ms
Speed: 0.0ms preprocess, 11.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 13.5ms
Speed: 0.0ms preprocess, 13.5ms inference, 1.0m

Speed: 1.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 11.1ms
Speed: 1.0ms preprocess, 11.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
t:  80%|█████████████████████████████████████████████████████▌             | 544/681 [00:20<00:04, 29.84it/s, now=None]
0: 384x640 (no detections), 10.3ms
Speed: 1.0ms preprocess, 10.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 14.0ms
Speed: 1.9ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 13.0ms
Speed: 1.1ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
t:  80%|█████████████████████████████████████████████████████▊             | 547/681 [00:20<00:04, 29.83it/s, now=None]
0: 384x640 (no detections), 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


Speed: 2.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 15.6ms
Speed: 2.0ms preprocess, 15.6ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
t:  94%|███████████████████████████████████████████████████████████████▏   | 642/681 [00:24<00:01, 25.64it/s, now=None]
0: 384x640 (no detections), 17.1ms
Speed: 2.1ms preprocess, 17.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 16.0ms
Speed: 1.5ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
t:  95%|███████████████████████████████████████████████████████████████▍   | 645/681 [00:24<00:01, 26.08it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./solidYellowLeft.mp4
CPU times: total: 3.22 s
Wall time: 25.6 s


In [15]:
#Creating empty video to write
out_video = './challenge.mp4'

#Loading our video
in_clip = VideoFileClip("test_videos/challenge.mp4")

#Sending each frame through our pipeline
out_clip = in_clip.fl_image(comp)

#Display progress
%time out_clip.write_videofile(out_video, audio = False)


0: 384x640 2 cars, 9.0ms
Speed: 0.5ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


Moviepy - Building video ./challenge.mp4.
Moviepy - Writing video ./challenge.mp4



t:   0%|                                                                             | 0/251 [00:00<?, ?it/s, now=None]
0: 384x640 2 cars, 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 15.4ms
Speed: 1.0ms preprocess, 15.4ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)
t:   1%|▊                                                                    | 3/251 [00:00<00:08, 29.31it/s, now=None]
0: 384x640 2 cars, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 17.7ms
Speed: 1.0ms preprocess, 17.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 13.8ms
Speed: 0.9ms preprocess, 13.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:   2%|█▋                                                                   | 6/251 [00:00<00:10, 23.12it/s, now=None]
0: 384x640 2 cars, 13.

t:  19%|█████████████                                                       | 48/251 [00:02<00:11, 18.27it/s, now=None]
0: 384x640 3 cars, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 17.3ms
Speed: 1.0ms preprocess, 17.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  20%|█████████████▌                                                      | 50/251 [00:02<00:11, 17.97it/s, now=None]
0: 384x640 2 cars, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
t:  21%|██████████████                                                      | 52/251 [00:02<00:11, 18.04it/s, now=None]
0: 384x640 2 cars, 16.7ms
Speed: 1.1ms preprocess, 16.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 16.


0: 384x640 2 cars, 1 boat, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)
t:  37%|█████████████████████████▍                                          | 94/251 [00:04<00:08, 19.35it/s, now=None]
0: 384x640 2 cars, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 16.5ms
Speed: 2.1ms preprocess, 16.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 17.1ms
Speed: 2.1ms preprocess, 17.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
t:  39%|██████████████████████████▎                                         | 97/251 [00:05<00:07, 19.43it/s, now=None]
0: 384x640 2 cars, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 1 train, 17.4ms
Speed: 1.1ms preprocess, 17.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
t

Slope for this frame could not be detected


Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 1 bus, 17.2ms
Speed: 1.0ms preprocess, 17.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
t:  51%|██████████████████████████████████▍                                | 129/251 [00:06<00:06, 18.97it/s, now=None]
0: 384x640 3 cars, 16.5ms
Speed: 1.0ms preprocess, 16.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  52%|██████████████████████████████████▉                                | 131/251 [00:06<00:06, 18.10it/s, now=None]
0: 384x640 2 cars, 16.6ms
Speed: 2.0ms preprocess, 16.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 17.1ms
Speed: 1.0ms preprocess, 17.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  53%|███████████████████████████████

Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 16.1ms
Speed: 2.0ms preprocess, 16.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  69%|██████████████████████████████████████████████▍                    | 174/251 [00:09<00:04, 17.68it/s, now=None]
0: 384x640 3 cars, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 15.9ms
Speed: 2.1ms preprocess, 15.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  70%|██████████████████████████████████████████████▉                    | 176/251 [00:09<00:04, 17.44it/s, now=None]
0: 384x640 2 cars, 17.1ms
Speed: 1.0ms preprocess, 17.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  71%|█████████████████████████████████████

Speed: 2.1ms preprocess, 16.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
t:  87%|██████████████████████████████████████████████████████████▏        | 218/251 [00:11<00:01, 18.25it/s, now=None]
0: 384x640 2 cars, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  88%|██████████████████████████████████████████████████████████▋        | 220/251 [00:11<00:01, 18.34it/s, now=None]
0: 384x640 2 cars, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
t:  88%|███████████████████████████████████████████████████████████▎       | 222/251 [00:12<00:01, 18.05it/s, now=None]
0: 384x640 2 cars, 16.0ms
Speed: 1.0ms preprocess

Moviepy - Done !
Moviepy - video ready ./challenge.mp4
CPU times: total: 4.28 s
Wall time: 14.1 s
